In [1]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
#Path para salvar los datos
path="H:/Mi unidad/pgd/consulta/plaza_publica"

In [ ]:
# === Configuración ===
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")

# Crear driver
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
todos = []

# ───────────────────────────────
# LOOP PRINCIPAL DE PÁGINAS
# ───────────────────────────────
for i in tqdm(range(1, 800), desc="Descargando páginas"):

    url = f"https://plazapublica.cdmx.gob.mx/processes/consultaPGD2025/f/14/meetings/{i}"

    try:
        driver.get(url)

        # Esperar contenedor de comentarios (AJAX)
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".comments"))
            )
        except:
            time.sleep(5)

        # Sacar HTML
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ───────────────────────────────
        # DATOS BÁSICOS DEL EVENTO
        # ───────────────────────────────
        try:
            nombre_evento = soup.find("h1", class_="h2 decorator").get_text(strip=True)
        except:
            nombre_evento = None

        try:
            lugar_evento = soup.find("div", class_="address__location").get_text(strip=True)
        except:
            lugar_evento = None

        try:
            direccion_evento = soup.find("div", class_="address__address").get_text(strip=True)
        except:
            direccion_evento = None

        # Fecha completa viene en data-tooltip
        span = soup.find("span", attrs={"data-tooltip": True})
        if span:
            tooltip_html = span.get("data-tooltip")
            tooltip_soup = BeautifulSoup(tooltip_html, "html.parser")
            fecha_completa = tooltip_soup.get_text(strip=True)
        else:
            fecha_completa = None

        # ───────────────────────────────
        # EXTRAER COMENTARIOS
        # ───────────────────────────────
        comentarios_html = soup.find_all("div", class_="comment")

        # Si NO hay comentarios, no agregamos nada
        if len(comentarios_html) == 0:
            continue

        # Barra de progreso interna para los comentarios
        for c in tqdm(comentarios_html, desc=f"Comentarios página {i}", leave=False):

            # Autor
            autor_tag = c.find("span", class_="author")
            autor = autor_tag.get_text(strip=True) if autor_tag else None

            # Fechas
            time_tag = c.find("time")
            fecha_visible = time_tag.get_text(strip=True) if time_tag else None
            fecha_iso = time_tag["datetime"] if time_tag and "datetime" in time_tag.attrs else None

            # Texto del comentario
            texto_tag = c.find("div", class_="comment__content")
            texto = texto_tag.get_text(strip=True) if texto_tag else None

            # ───────────────────────────────
            # GUARDAR REGISTRO
            # ───────────────────────────────
            if texto and texto.strip() != "":
                todos.append({
                    "id_taller": i,
                    "nombre_evento": nombre_evento,
                    "lugar_evento": lugar_evento,
                    "direccion_evento": direccion_evento,
                    "fecha_evento": fecha_completa,
                    "autor": autor,
                    "fecha_visible": fecha_visible,
                    "fecha_iso": fecha_iso,
                    "texto": texto
                })

    except Exception as e:
        tqdm.write(f"Error en página {i}: {e}")
        continue

# Cerrar driver
driver.quit()


Descargando páginas:   0%|          | 0/799 [00:00<?, ?it/s]

Descargando páginas: 100%|██████████| 799/799 [3:14:17<00:00, 14.59s/it]  


In [5]:
# Pasar a df
coment_talleres = pd.DataFrame(todos)
coment_talleres

,id_taller,nombre_evento,lugar_evento,direccion_evento,fecha_evento,autor,fecha_visible,fecha_iso,texto
0,42,"Taller ""de los problemas a las propuestas""- PI...",- PILARES Héroes de Cerro Prieto,"C. Rublos 57, Héroes de Cerro Prieto, Gustavo ...",11-11-2025 12:00 PM CST (GMT -06:00),Julio López García,11/11/2025 09:49,2025-11-11T09:49:34-06:00,La participación ciudadana ahora es una practi...
1,43,"Taller ""de los problemas a las propuestas""- PI...",PILARES Héroes de Cerro Prieto,"C. Rublos 57, Héroes de Cerro Prieto, Gustavo ...",25-11-2025 12:00 PM CST (GMT -06:00),Julio López García,11/11/2025 09:48,2025-11-11T09:48:12-06:00,Es una gran oportunidad para dar a conocer el ...
2,56,"Taller ""de los problemas a las propuestas""- PI...",PILARES Paquita Calvo Zapata,"Av. Lenguas Indígenas 31, Carlos Zapata Vela, ...",11-11-2025 11:00 AM CST (GMT -06:00),Roberto González,12/11/2025 13:41,2025-11-12T13:41:17-06:00,SUMAMENTE DEFICIENTE EL TALLER.Soy representa...
3,318,"Taller ""de los problemas a las propuestas"" - ...",PILARES Villa Panamericana,"Av. Panamericana S/N, Pedregal de Carrasco Sec...",02-12-2025 16:00 PM CST (GMT -06:00),Ernesto Vargas,30/11/2025 18:42,2025-11-30T18:42:34-06:00,Agradezco la oportunidad de participar como es...


In [6]:
###Salvar a csv
coment_talleres.to_csv(path+"/comen_taller_plaza_publica.csv", index=False, encoding="utf-8-sig")